In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Aplicacion con API incluida

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import joblib
import os

# Configuraciones iniciales
img_size = 128
batch_size = 32

# Rutas de los modelos y datos
model_dir = '/content/drive/MyDrive/BOOTCAMP_DATA_SCIENCE/PROYECTO_7'

# Función para cargar modelos de manera segura
def load_keras_model(path):
    try:
        model = load_model(path)
        print(f"Model loaded successfully from {path}")
        return model
    except Exception as e:
        print(f"Failed to load model from {path}. Error: {e}")
        return None

# Cargar modelos
model_simple = load_keras_model(f'{model_dir}/final_model_simple.h5')
model_cnn = load_keras_model(f'{model_dir}/final_model_cnn.h5')
model_vgg16 = load_keras_model(f'{model_dir}/final_model_vgg16.h5')

try:
    meta_model = joblib.load(f'{model_dir}/saved_models/meta_model.h5')
    print("Meta-model loaded successfully.")
except Exception as e:
    print(f"Failed to load meta-model. Error: {e}")
    meta_model = None

def make_predictions(file_path):
    """Procesa una imagen y usa los modelos para hacer una predicción."""
    try:
        img = image.load_img(file_path, target_size=(img_size, img_size))
        img_array = image.img_to_array(img)
        img_batch = np.expand_dims(img_array, axis=0)
        img_preprocessed = img_batch / 255.0

        pred_simple = model_simple.predict(img_preprocessed) if model_simple else None
        pred_cnn = model_cnn.predict(img_preprocessed) if model_cnn else None
        pred_vgg16 = model_vgg16.predict(img_preprocessed) if model_vgg16 else None
        stacked_features = np.concatenate([x for x in [pred_simple, pred_cnn, pred_vgg16] if x is not None], axis=1)

        if meta_model:
            final_prediction = meta_model.predict(stacked_features)
            return 'Pneumonia' if final_prediction[0] == 1 else 'Normal'
        else:
            return "Meta-model is not loaded."
    except Exception as e:
        return f"An error occurred during prediction: {e}"

if __name__ == '__main__':
    # Código de prueba para verificar la funcionalidad del módulo
    test_image_path = '/content/drive/MyDrive/BOOTCAMP_DATA_SCIENCE/PROYECTO_7/DATA_CLEAN_BALANCED/val/pneumonia/person1946_bacteria_4874.jpeg'
    try:
        result = make_predictions(test_image_path)
        print(f'Prediction for the test image is: {result}')
    except Exception as e:
        print(f"An error occurred: {e}")


Model loaded successfully from /content/drive/MyDrive/BOOTCAMP_DATA_SCIENCE/PROYECTO_7/final_model_simple.h5
Model loaded successfully from /content/drive/MyDrive/BOOTCAMP_DATA_SCIENCE/PROYECTO_7/final_model_cnn.h5
Model loaded successfully from /content/drive/MyDrive/BOOTCAMP_DATA_SCIENCE/PROYECTO_7/final_model_vgg16.h5
Meta-model loaded successfully.
1/1 [==============================] - 1s 944ms/step


1/1 [==============================] - 0s 169ms/step


1/1 [==============================] - 0s 319ms/step
Prediction for the test image is: An error occurred during prediction: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.


#Aplicación

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import joblib

# Configuraciones iniciales
img_size = 128
batch_size = 32

# Rutas de los modelos y datos
model_dir = '/content/drive/MyDrive/BOOTCAMP_DATA_SCIENCE/PROYECTO_7'

# Cargar modelos
model_simple = load_model(f'{model_dir}/final_model_simple.h5')
model_cnn = load_model(f'{model_dir}/final_model_cnn.h5')
model_vgg16 = load_model(f'{model_dir}/final_model_vgg16.h5')
meta_model = joblib.load(f'{model_dir}/saved_models/meta_model.h5')

def make_predictions(image_path):
    """Procesa una imagen y usa los modelos para hacer una predicción."""
    test_datagen = ImageDataGenerator(rescale=1./255)
    image = test_datagen.flow_from_directory(
        image_path,
        target_size=(img_size, img_size),
        batch_size=1,
        class_mode='binary',
        shuffle=False
    )
    pred_simple = model_simple.predict(image)
    pred_cnn = model_cnn.predict(image)
    pred_vgg16 = model_vgg16.predict(image)
    stacked_features = np.concatenate([pred_simple, pred_cnn, pred_vgg16], axis=1)
    final_prediction = meta_model.predict(stacked_features)
    return 'Pneumonia' if final_prediction[0] == 1 else 'Normal'


#API

In [2]:
!pip install flask-ngrok

In [3]:
!pip install pyngrok

In [5]:
def make_predictions(file_path):
    """Procesa una imagen y usa los modelos para hacer una predicción."""
    from tensorflow.keras.preprocessing import image
    img = image.load_img(file_path, target_size=(img_size, img_size))
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    img_preprocessed = img_batch / 255.0

    pred_simple = model_simple.predict(img_preprocessed)
    pred_cnn = model_cnn.predict(img_preprocessed)
    pred_vgg16 = model_vgg16.predict(img_preprocessed)
    stacked_features = np.concatenate([pred_simple, pred_cnn, pred_vgg16], axis=1)
    final_prediction = meta_model.predict(stacked_features)
    return 'Pneumonia' if final_prediction[0] == 1 else 'Normal'


In [6]:
from pyngrok import ngrok

# Opcional: Configura tu token de ngrok (esto es recomendable para un uso más estable)
# ngrok.set_auth_token("tu_token_de_ngrok")

# Opcional: Configura tu token de ngrok (esto es recomendable para un uso más estable)
ngrok.set_auth_token("2ifROKcb0VRbpopQpwC4cyhnfhr_49M4Kf7F6kDdd4dhu1wsc")

# Detener todos los túneles existentes antes de crear uno nuevo
ngrok.kill()

# Establecer un túnel en el puerto 5000
tunnel = ngrok.connect(5000)
print("Ngrok Tunnel URL:", tunnel.public_url)


Ngrok Tunnel URL: https://dc3e-34-75-110-56.ngrok-free.app


In [7]:
print(ngrok.get_tunnels())

[<NgrokTunnel: "https://dc3e-34-75-110-56.ngrok-free.app" -> "http://localhost:5000">]


In [8]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    # Aquí iría la lógica para manejar la solicitud POST
    return jsonify({'message': 'Esta es la respuesta del endpoint /predict'})

if __name__ == '__main__':
    app.run(port=5000)  # Flask se ejecuta en el puerto 5000


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Jul/2024 04:21:31] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Jul/2024 04:21:31] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Jul/2024 04:21:43] "GET / HTTP/1.1" 404 -


In [10]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    # Aquí iría la lógica para manejar la solicitud POST
    return jsonify({'message': 'Esta es la respuesta del endpoint /predict'})

if __name__ == '__main__':
    run_with_ngrok(app)  # Use run_with_ngrok to expose the app
    app.run()  # Flask will now be accessible via ngrok

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-16:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [ ]:
from flask import Flask, request, jsonify
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)

# Configuraciones para la carga de archivos
app.config['UPLOAD_FOLDER'] = '/tmp'  # Define una carpeta temporal para guardar los archivos
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # Limita el tamaño del archivo a 16MB

@app.route('/predict', methods=['POST'])
def predict():
    # Verificar si el archivo parte existe en la solicitud
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        # Usar la función de predicción con la ruta del archivo guardado
        prediction = make_predictions(file_path)
        return jsonify({'prediction': prediction})

    return jsonify({'error': 'File not allowed'}), 400

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
